# OCRテスト

- 読み取りたい文字がある領域を切り抜いて加工せずOCR

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from pathlib import Path

import pandas as pd
from PIL import Image, ImageDraw, ImageFont


In [10]:
!pip install paddlepaddle-gpu -i https://mirror.baidu.com/pypi/simple

!pip install "paddleocr>=2.0.1" # Recommend to use version 2.0.1+


Looking in indexes: https://mirror.baidu.com/pypi/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 393.8 MB 26 kB/s 
     |████████████████████████████████| 394 kB 13.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
!wget https://github.com/Halfish/lstm-ctc-ocr/raw/master/fonts/simfang.ttf

--2022-07-09 11:50:46--  https://github.com/Halfish/lstm-ctc-ocr/raw/master/fonts/simfang.ttf
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Halfish/lstm-ctc-ocr/master/fonts/simfang.ttf [following]
--2022-07-09 11:50:46--  https://raw.githubusercontent.com/Halfish/lstm-ctc-ocr/master/fonts/simfang.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10576012 (10M) [application/octet-stream]
Saving to: ‘simfang.ttf.1’

simfang.ttf.1       100%[===================>]  10.09M  --.-KB/s    in 0.07s   

2022-07-09 11:50:46 (153 MB/s) - ‘simfang.ttf.1’ saved [10576012/10576012]



In [12]:
path_input_root = Path('/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped')
print(path_input_root)

/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped


In [13]:
for img_file in path_input_root.glob('*'):
  print(img_file)
  

/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281029_A
/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281032_A
/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281037_A
/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281040_A
/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281043_A
/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281046_A
/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281051_A
/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281055_A
/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281101_A
/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281109_A
/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281116_A
/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281127_A
/con

In [14]:
from paddleocr import PaddleOCR, draw_ocr
from paddleocr.tools.infer.utility import draw_ocr_box_txt
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.


/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sc

In [15]:
import cv2
from PIL import Image

In [ ]:
det_algo = 'DB'

ocr = PaddleOCR(
    use_angle_cls=False, lang='en', det_algorithm=det_algo, 
    det_db_thresh=0.01, det_db_box_thresh=0.01, cls_thresh=0.01, det_pse_box_thresh=0.01,
    drop_score=0.00
) # need to run only once to download and load model into memory

scales = [2, 4, 8]

df_summary = pd.DataFrame(columns=['file', 'idx', 'Num of detected'])
summaries = list()
for img_dir in path_input_root.glob('*'):
  print(img_dir)
  for idx, img_file in enumerate(img_dir.glob('*.png')):
    for scale in scales:
      img = Image.open(str(img_file))
      (width, height) = (img.width // scale, img.height // scale)
      # 画像をリサイズする
      img_resized = img.resize((width, height))  # TODO: height,width逆
      # ファイルを保存
      scaled_file_path = img_dir/f'{img_file.stem}_resized_{scale}.png'
      img_resized.save(str(scaled_file_path), quality=100)
      summary = {
          'file': img_dir.stem,
          'idx': idx,
          'scale': scale
      }
      print(f'idx: {idx}, scale: {scale}')

      result = ocr.ocr(str(img_file), cls=False)

      for line in result:
          print(line)
      # draw result
      image = Image.open(str(scaled_file_path)).convert('RGB')
      image_cv = cv2.imread(str(scaled_file_path))
      # print(image_cv.shape)
      print(image_cv)
      #image = answer1
      boxes = [line[0] for line in result]  # [upper_left, upper_right, lower_left, lower_right]
      txts = [line[1][0] for line in result]
      scores = [line[1][1] for line in result]

      summary[f'Num of detected'] = len(boxes)
      print(f"number of objects: {len(boxes)}")

      image_d = ImageDraw.Draw(image)
      font = ImageFont.truetype('/content/simfang.ttf', size=20)
      if boxes and len(boxes[0]):
        for box, text in zip(boxes, txts):
          if not len(text):
            continue
          print(box)
          image_d.rectangle(
              [tuple(box[0]), tuple(box[2])], outline=(255, 0, 0), width=5
          )
          image_d.text(tuple([box[0][0], box[0][1]-20]), text, fill=(255, 0, 0), font=font)
      path_output_cropped_dir = Path(f'/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/output/04_cropped_ocr_resize/{img_dir.stem}')
      path_output_cropped_dir.mkdir(parents=True, exist_ok=True)
      image.save(path_output_cropped_dir/f'direct_{img_dir.stem}_ocr_{det_algo}_idx_{idx}_scale_{scale}.png')

      im_show = draw_ocr(image, boxes, txts, scores, font_path='/content/simfang.ttf', drop_score=0.30)
      im_show = Image.fromarray(im_show)
      im_show.save(path_output_cropped_dir/f'{img_dir.stem}_ocr_{det_algo}_idx_{idx}_scale_{scale}.png')
      summaries.append(summary)
  print('=' * 100)

df_summary = pd.DataFrame(summaries)
path_output_root_dir = Path(f'/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/output/03_cropped_ocr')
df_summary.to_csv(path_output_root_dir/'summary.csv', index=False, header=True)

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3.86M/3.86M [00:50<00:00, 76.1kiB/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer/en_PP-OCRv3_rec_infer.tar


100%|██████████| 9.96M/9.96M [00:13<00:00, 721kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 1.45M/1.45M [00:07<00:00, 184kiB/s]


Streaming output truncated to the last 5000 lines.
 [[209 209 209]
  [207 207 207]
  [209 209 209]
  ...
  [193 193 193]
  [192 192 192]
  [186 186 186]]

 [[185 185 185]
  [184 184 184]
  [183 183 183]
  ...
  [153 153 153]
  [156 156 156]
  [167 167 167]]

 [[225 225 225]
  [224 224 225]
  [223 223 223]
  ...
  [205 205 205]
  [207 207 207]
  [209 209 209]]]
number of objects: 6
[[439.0, 82.0], [446.0, 82.0], [446.0, 89.0], [439.0, 89.0]]
[[559.0, 229.0], [588.0, 229.0], [588.0, 239.0], [559.0, 239.0]]
/content/drive/MyDrive/プロジェクト/pj_bridgestone_ocr/input/cropped/tire_2202281508_A
idx: 0, scale: 2
[2022/07/09 11:53:18] ppocr DEBUG: dt_boxes num : 0, elapse : 0.006905078887939453
[2022/07/09 11:53:18] ppocr DEBUG: rec_res num  : 0, elapse : 1.430511474609375e-06
[[[231 231 231]
  [246 246 246]
  [228 228 227]
  ...
  [244 244 244]
  [255 255 255]
  [248 248 248]]

 [[238 238 238]
  [245 246 245]
  [245 245 245]
  ...
  [246 247 246]
  [240 239 239]
  [248 249 249]]

 [[245 244 245]